# Chrun Prediction

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [53]:
df = pd.read_csv("Churn_Modelling.csv")

In [54]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Preprocess the data**

In [55]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [56]:
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


**Encode**

In [57]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [58]:
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


Onehot encode Geography

In [59]:
from sklearn.preprocessing import OneHotEncoder

In [60]:
onehot_geo = OneHotEncoder(sparse_output=False)
geo_encoder = onehot_geo.fit_transform(df[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [61]:
onehot_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [62]:
encoded_geography = pd.DataFrame(geo_encoder,columns=onehot_geo.get_feature_names_out(['Geography']))
encoded_geography.head(3)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0


Combine Columns

In [63]:
data = pd.concat([df,encoded_geography],axis=1)
data.drop('Geography',axis=1,inplace=True)
data.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0


Split data

In [64]:
X = data.drop('Exited',axis=1)
y = data['Exited']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Scale data

In [65]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

Save encoder and scaler

In [66]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geography.pkl','wb') as file:
    pickle.dump(onehot_geo,file)
with open('scale.pkl','wb') as file:
    pickle.dump(scaler,file)

Train ANN

In [67]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Activation,BatchNormalization,Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [95]:
import logging
import os
import sys
import warnings
from urllib.parse import urlparse

import dagshub
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
import webbrowser
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [98]:
MLFLOW_TRACKING_URI = "https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow"

In [97]:
TOKEN = dagshub.auth.get_token()
os.environ["MLFLOW_TRACKING_USERNAME"] = "arjunravi726"
os.environ["MLFLOW_TRACKING_PASSWORD"] = TOKEN

In [99]:
webbrowser.open(MLFLOW_TRACKING_URI)

True

In [69]:
dagshub.init(repo_name='Customer-Churn-Prediction',repo_owner='arjunravi726',mlflow=True)

Initialized MLflow to track repo "arjunravi726/Customer-Churn-Prediction"

Repository arjunravi726/Customer-Churn-Prediction initialized!

In [70]:
# mlflow.log_metric('training_loss',)

In [100]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [101]:
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

Build ANN model

In [74]:
X.shape[1]

12

In [75]:
model = Sequential([
    Input(shape=(X.shape[1],)),
    BatchNormalization(),
    Dense(64),
    BatchNormalization(),
    Activation('relu'),
    Dropout(rate=0.5),
    Dense(32),
    Activation('relu'),
    Dropout(rate=0.5),
    Dense(1, activation='sigmoid')
])

In [76]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 12)             │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,249 (12.69 KB)

 Trainable params: 3,097 (12.10 KB)

 Non-trainable params: 152 (608.00 B)

In [77]:
from tensorflow.keras.callbacks import Callback

In [78]:
class CustomCallbacks(Callback):
    def on_epoch_end(self,epoch,logs=None):
        if logs['val_loss'] < 0.5 or logs['val_accuracy'] > 0.9:
            self.model.stop_training = False

In [79]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [80]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

Set up Tensorboard

In [81]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [82]:
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

Early Stopping

In [83]:
early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [84]:
history = model.fit(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=100,callbacks=[tensorflow_callback,CustomCallbacks()])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.7803 - loss: 0.5122 - val_accuracy: 0.8265 - val_loss: 0.4008
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8195 - loss: 0.4323 - val_accuracy: 0.8530 - val_loss: 0.3958
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8241 - loss: 0.4243 - val_accuracy: 0.8590 - val_loss: 0.3496
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8351 - loss: 0.3985 - val_accuracy: 0.8535 - val_loss: 0.3570
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8374 - loss: 0.3905 - val_accuracy: 0.8595 - val_loss: 0.3508
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8335 - loss: 0.3921 - val_accuracy: 0.8445 - val_loss: 0.3609
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8391 - loss: 0.3885 - val_accuracy: 0.8595 - val_loss: 0.3396
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8396 - loss: 0.3805 - val_acc

In [86]:
# mlflow.sklearn.log_model(model)

In [92]:
import mlflow
with mlflow.start_run():
  # Your training code here...
  mlflow.log_metric('train_acc', history.history['accuracy'][-1])
  mlflow.log_param('val_acc',history.history['val_accuracy'][-1])
  mlflow.log_param('val_loss',history.history['val_loss'][-1])
  mlflow.log_param('loss',history.history['loss'][-1])

🏃 View run thundering-hen-476 at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0/runs/d5cd9ff263684575bde00d6f80d0ecb3
🧪 View experiment at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0


In [91]:
# mlflow.end_run()

🏃 View run debonair-shad-746 at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0/runs/ba47d884708c4970a83a3b230e902347
🧪 View experiment at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0


In [26]:
# import keras
# keras.saving.save_model(model, 'model.keras')

In [27]:
model.save('model.h5')

Load Tensorboard Extension

In [52]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [53]:
!kill 6460

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [54]:
%tensorboard --logdir logs/fit20241228-160457

Reusing TensorBoard on port 6006 (pid 12184), started 0:02:33 ago. (Use '!kill 12184' to kill it.)

Prediction

In [31]:
from tensorflow.keras.models import load_model

In [32]:
loaded_model = load_model('model.h5')

In [33]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

In [34]:
with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_geo = pickle.load(file)

In [35]:
with open('scale.pkl','rb') as file:
    scaler = pickle.load(file)

In [36]:
input_data = {
    'CreditScore':600,
    'Geography': 'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard': 1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [37]:
df = pd.DataFrame([input_data])

In [38]:
label_encoder_gender.transform(df['Gender'])

array([1])

In [39]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])

In [40]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [41]:
geo_encoded= onehot_geo.transform([df['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_geo.get_feature_names_out())

d:\BroCamp\Deep Learning\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [42]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [43]:
df = pd.concat([df,geo_encoded_df],axis=1)

In [44]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [45]:
df.drop('Geography',axis=1,inplace=True)

In [ ]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [47]:
df_scaled = scaler.transform(df)

In [48]:
df_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [49]:
prediction_prob = loaded_model.predict(df_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step


In [50]:
if prediction_prob > 0.5:
    print('The Customer is likely to churn')
else:
    print('The Customer is not likely to churn')

The Customer is not likely to churn


In [51]:
onehot_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

Intergrate with streamlit